# Question 1

scala> spark.version
res0: String = 3.3.2

# Question 2

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/06 16:06:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/06 16:06:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [16]:
schema = types.StructType([
    #types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [17]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-06.csv.gz')

In [18]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [21]:
df = df.repartition(12)
df.write.mode('overwrite').parquet('fhvhv/2021/06/')
df = spark.read.parquet('fhvhv/2021/06/')
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [22]:
!ls -l --block-size=M fhvhv/2021/06

total 284M
-rw-r--r-- 1 Brian Brian  0M Mar  6 16:44 _SUCCESS
-rw-r--r-- 1 Brian Brian 24M Mar  6 16:44 part-00000-a2b24d6a-811f-41e7-9a24-fdc6600ef9ea-c000.snappy.parquet
-rw-r--r-- 1 Brian Brian 24M Mar  6 16:44 part-00001-a2b24d6a-811f-41e7-9a24-fdc6600ef9ea-c000.snappy.parquet
-rw-r--r-- 1 Brian Brian 24M Mar  6 16:44 part-00002-a2b24d6a-811f-41e7-9a24-fdc6600ef9ea-c000.snappy.parquet
-rw-r--r-- 1 Brian Brian 24M Mar  6 16:44 part-00003-a2b24d6a-811f-41e7-9a24-fdc6600ef9ea-c000.snappy.parquet
-rw-r--r-- 1 Brian Brian 24M Mar  6 16:44 part-00004-a2b24d6a-811f-41e7-9a24-fdc6600ef9ea-c000.snappy.parquet
-rw-r--r-- 1 Brian Brian 24M Mar  6 16:44 part-00005-a2b24d6a-811f-41e7-9a24-fdc6600ef9ea-c000.snappy.parquet
-rw-r--r-- 1 Brian Brian 24M Mar  6 16:44 part-00006-a2b24d6a-811f-41e7-9a24-fdc6600ef9ea-c000.snappy.parquet
-rw-r--r-- 1 Brian Brian 24M Mar  6 16:44 part-00007-a2b24d6a-811f-41e7-9a24-fdc6600ef9ea-c000.snappy.parquet
-rw-r--r-- 1 Brian Brian 24M Mar  6 16:44 part-00008-a2b24

In [23]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02889|2021-06-04 20:51:44|2021-06-04 21:10:12|         239|         158|      N|                B02889|
|              B02800|2021-06-04 15:50:15|2021-06-04 16:19:29|          75|         116|      N|                  null|
|              B02510|2021-06-02 21:03:38|2021-06-02 21:10:12|         167|         168|      N|                  null|
|              B02867|2021-06-02 12:51:57|2021-06-02 13:05:09|         151|         142|      N|                B02867|
|              B02869|2021-06-21 09:51:45|2021-06-21 10:09:17|         106|          65|      N|                B02869|
|              B02764|2021-06-02 13:27:0

# Question 3

In [24]:
df.registerTempTable('trips_data')

In [25]:
spark.sql("""
SELECT
    count(1)
FROM
    trips_data
WHERE
    CAST(pickup_datetime as DATE) = '2021-06-15'
""").show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



# Question 4

In [27]:
spark.sql("""
SELECT
    MAX(unix_timestamp(dropoff_datetime)-unix_timestamp(pickup_datetime)) / 3600
FROM
    trips_data
WHERE
""").show()

+----------------------------------------------------------------------------------------------------------------------------+
|(max((unix_timestamp(dropoff_datetime, yyyy-MM-dd HH:mm:ss) - unix_timestamp(pickup_datetime, yyyy-MM-dd HH:mm:ss))) / 3600)|
+----------------------------------------------------------------------------------------------------------------------------+
|                                                                                                            66.8788888888889|
+----------------------------------------------------------------------------------------------------------------------------+



# Question 6

In [28]:
df_zones = spark.read.parquet('zones/')

In [29]:
df_zones.registerTempTable('zones')

In [30]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [32]:
spark.sql("""
SELECT
    zones.Zone,
    COUNT(1) as trips
FROM
    trips_data
INNER JOIN
    zones
ON
    trips_data.PULocationID = zones.LocationID
GROUP BY
    zones.Zone
ORDER BY 
    trips DESC
""").show()

+--------------------+------+
|                Zone| trips|
+--------------------+------+
| Crown Heights North|231279|
|        East Village|221244|
|         JFK Airport|188867|
|      Bushwick South|187929|
|       East New York|186780|
|TriBeCa/Civic Center|164344|
|   LaGuardia Airport|161596|
|            Union Sq|158937|
|        West Village|154698|
|             Astoria|152493|
|     Lower East Side|151020|
|        East Chelsea|147673|
|Central Harlem North|146402|
|Williamsburg (Nor...|143683|
|          Park Slope|143594|
|  Stuyvesant Heights|141427|
|        Clinton East|139611|
|West Chelsea/Huds...|139431|
|             Bedford|138428|
|         Murray Hill|137879|
+--------------------+------+
only showing top 20 rows

